In [ ]:
import sys
sys.path.insert(1, '../scripts')

: 

In [2]:
import pandas as pd

# import custom scripts
import fractional_charge as fc
import charge_distribution as cd

In [3]:
df = pd.read_csv('../../data/charged_regions/cr_raw.csv', comment='#')
df.head(5)

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,kappa1,kappa2,uni_id,orf_label
0,YAL011W,SWC3,626,0,54,MPAVLRTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDD...,55,0.015674,0.527273,0.297167,0.616928,P31376,verified
1,YAL011W,SWC3,626,169,265,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,97,0.050753,0.536082,0.051716,0.110243,P31376,verified
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,0.044964,0.096912,P31376,verified
3,YAL011W,SWC3,626,470,525,KSVVEFLEDTDEIIISWIVIHNSKEIEKFKTKKIKAKLKADQKLNK...,56,0.000000,0.428571,0.114622,0.300249,P31376,verified
4,YAL013W,DEP1,406,81,163,TLTESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAI...,83,0.141633,0.542169,0.093375,0.239955,P31385,verified


In [10]:
colnames = list(df.columns)

In [5]:
def trim_region(seq):
    charged_AAs = ['K', 'R', 'D', 'E']
    n_lefttrim, n_righttrim = 0, 0
    for AA in seq:
        if AA not in charged_AAs:
            n_lefttrim += 1
        else:
            break
    for AA in seq[::-1]:
        if AA not in charged_AAs:
            n_righttrim += 1
        else:
            break
    trimmed_seq = seq[n_lefttrim:(len(seq) - n_righttrim)]
    return trimmed_seq, n_lefttrim, n_righttrim


In [6]:
testseq = df.iloc[0]['region.seq']
print(testseq)
print(trim_region(testseq))

MPAVLRTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDDAYDEVGNDY
('RTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDDAYDEVGND', 5, 1)


In [7]:
testseq2 = df.iloc[1]['region.seq']
print(testseq2)
print(trim_region(testseq2))

RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQQMQKKIAKEQKLQLQKENKAKQKLEQEALKLKRKEEMKKLKEQNKNKQGSP
('RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQQMQKKIAKEQKLQLQKENKAKQKLEQEALKLKRKEEMKKLKEQNKNK', 0, 4)


In [8]:
testseq3 = df.iloc[2]['region.seq']
print(testseq3)
print(trim_region(testseq3))

KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEKGVKSEDTQKKEDNQVVPK
('KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEKGVKSEDTQKKEDNQVVPK', 0, 0)


In [11]:
dict_trimmed = []
for index, row in df.iterrows():
    rv = {}
    for name in colnames:
        rv[name] = row[name]
    oldseq = rv['region.seq']
    newseq, n_lefttrim, n_righttrim = trim_region(oldseq)
    rv['left.bound'] = row['left.bound'] + n_lefttrim
    rv['right.bound'] = row['right.bound'] - n_righttrim
    rv['region.seq'] = newseq
    rv['region.len'] = row['region.len'] - (n_lefttrim + n_righttrim)
    rv['charge.asymmetry'] = cd.get_sigma(newseq)
    rv['frac.charge'] = fc.get_fractional_charge(newseq)
    dict_trimmed.append(rv)
df_trimmed = pd.DataFrame.from_records(dict_trimmed)

In [12]:
df_trimmed.head(5)

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,kappa1,kappa2,uni_id,orf_label
0,YAL011W,SWC3,626,5,53,RTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDDAYDEVGND,49,0.017593,0.591837,0.297167,0.616928,P31376,verified
1,YAL011W,SWC3,626,169,261,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,93,0.052936,0.559140,0.051716,0.110243,P31376,verified
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,0.044964,0.096912,P31376,verified
3,YAL011W,SWC3,626,470,525,KSVVEFLEDTDEIIISWIVIHNSKEIEKFKTKKIKAKLKADQKLNK...,56,0.000714,0.428571,0.114622,0.300249,P31376,verified
4,YAL013W,DEP1,406,84,159,ESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAIDSK...,76,0.138444,0.592105,0.093375,0.239955,P31385,verified


In [ ]:
df_trimmed.to_csv('.')